In [1]:
import numpy as np
import pandas as pd
import time
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, KNNBasic
from surprise import accuracy
from surprise.model_selection import KFold

In [2]:
df1 = pd.read_csv('./combined_data_1.txt', header = None, names = ['user', 'rating'], usecols = [0,1])
# df2 = pd.read_csv('./combined_data_2.txt', header = None, names = ['user', 'rating'], usecols = [0,1])
# df3 = pd.read_csv('./combined_data_3.txt', header = None, names = ['user', 'rating'], usecols = [0,1])
# df4 = pd.read_csv('./combined_data_4.txt', header = None, names = ['user', 'rating'], usecols = [0,1])

df1['rating'] = df1['rating'].astype(float)
# df2['rating'] = df2['rating'].astype(float)
# df3['rating'] = df3['rating'].astype(float)
# df4['rating'] = df4['rating'].astype(float)

df = df1
# df = df1.append(df2)
# df = df.append(df3)
# df = df.append(df4)
df.index = np.arange(0,len(df))
df.head()

,user,rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


In [3]:
df_nan = pd.DataFrame(pd.isnull(df.rating))
df_nan = df_nan[df_nan['rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

print('Movie numpy: {}'.format(movie_np))
print('Length: {}'.format(len(movie_np)))
df = df[pd.notnull(df['rating'])]

df['item'] = movie_np.astype(int)
df['user'] = df['user'].astype(int)
print('-Dataset examples-')
print(df.iloc[::5000000, :])

Movie numpy: [1.000e+00 1.000e+00 1.000e+00 ... 4.499e+03 4.499e+03 4.499e+03]
Length: 24053764


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


-Dataset examples-
             user  rating  item
1         1488844     3.0     1
5000996    501954     2.0   996
10001962   404654     5.0  1962
15002876   886608     2.0  2876
20003825  1193835     2.0  3825


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
probe = pd.read_csv('./probe.txt', names = ['user'], usecols = [0], header = None)
probe['item_col'] = probe['user'].map(lambda x:list(x)[-1]==':')

temp_probe = probe[probe['item_col']==True]
temp_probe = temp_probe.reset_index()

movie_np = []
movie_id_index = 0

for i,j in zip(temp_probe['index'][1:],temp_probe['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), temp_probe.user[movie_id_index])
    movie_np = np.append(movie_np, temp)
    movie_id_index += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(probe) - temp_probe.iloc[-1, 0] - 1),temp_probe.user[movie_id_index])
movie_np = np.append(movie_np, last_record)

probe = probe[probe['item_col']==False]

probe['item'] = movie_np
probe['item'] = probe['item'].map(lambda x:x[:-1]).astype(int)
probe['user'] = probe['user'].astype(int)

probe = probe.drop(columns='item_col')
probe.head()

,user,item
1,30878,1
2,2647871,1
3,1283744,1
4,2488120,1
5,317050,1


In [5]:
probe['probe'] = 1
df = pd.merge(df,probe,how='left',on=['user','item'])

new_df = df[~(df['probe']==1)][['user','item','rating']]
new_probe = df[df['probe']==1][['user','item','rating']]
new_df.shape[0],new_probe.shape[0]

(23476551, 577213)

In [12]:
reader = Reader(line_format='user item rating timestamp',rating_scale=(1, 5))
from surprise.model_selection import train_test_split

train = Dataset.load_from_df(new_df.iloc[:11000000,:],reader=reader)
test = Dataset.load_from_df(new_probe,reader=reader)

bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
algo = BaselineOnly(bsl_options=bsl_options)


train_probe, test_probe = train_test_split(test,test_size=.95)
algo.fit(train.build_full_trainset())
predictions = algo.test(test_probe)

# Then compute RMSE
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 1.0665


1.0664826104102805

In [18]:
len(test_probe)

548353